In [1]:
!pip install pathway bokeh panel --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Importing Necessary Libraries

In [2]:
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn

### Importing Dataset

In [3]:
df=pd.read_csv('dataset.csv')
df=df.rename(columns={'SystemCodeNumber':'LotID'})

### Creating Timestamp by Combining the Columns

In [4]:
df['Timestamp']=pd.to_datetime(df['LastUpdatedDate']+ " " +df['LastUpdatedTime'],format="%d-%m-%Y %H:%M:%S")

In [5]:
df = df[["Timestamp", "LotID", "Occupancy", "Capacity"]]
df.to_csv("parking_stream_m1.csv", index=False)

### Pathway Streaming Schema

In [6]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    LotID: str
    Occupancy: int
    Capacity: int

### Simulating Real Time Stream

In [7]:
data = pw.demo.replay_csv("parking_stream_m1.csv", schema=ParkingSchema, input_rate=1000)
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    LotID = data.LotID
)

### Model 1 Logic

In [8]:
BASE_PRICE = 10.0
ALPHA = 2.0

In [9]:
@pw.udf
def baseline_price(occupancy: int, capacity: int) -> float:
    if capacity == 0:
        return BASE_PRICE
    price = BASE_PRICE + ALPHA * (occupancy / capacity)
    return max(5, min(20, round(price, 2)))

price_stream = data_with_time.with_columns(
    price = baseline_price(
        data_with_time.Occupancy,
        data_with_time.Capacity
    )
)

### Bokeh Plot

In [10]:
pn.extension()
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400, width=800, title="Model 1: Baseline Linear Pricing",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="blue")
    fig.scatter("t", "price", source=source, size=6, color="green")
    return fig

viz = price_stream.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
import sys
import contextlib
with contextlib.redirect_stdout(sys.stdout):
    pw.run()

Output()